# Survey analysis
Analyse the results from the [languages survey](https://forms.gle/5b3mZRVcgAsoNG1FA)

In [1]:
all_langs = ['Python', 'Java', 'JavaScript', 'TypeScript', 'PHP', 'SQL', 'C', 'C++', 'C#',
             'Ruby', 'R', 'Matlab', 'Go', 'Rust', 'Objective-C', 'Swift', 'Visual Basic',
             'Perl', 'Cobol', 'Fortran', 'Lisp', 'Assembly', 'Kotlin', 'Dart', 'Scala',
             'Lua', 'Delphi', 'Haskell', 'Julia', 'Clojure', 'Elixir', 'Pascal']

print(len(all_langs))

32


## Load data

In [2]:
import pandas as pd


df = pd.read_csv("../data/Programming language survey.csv", header=0, names=("timestamp", "languages", "years"), usecols=("languages", "years"))
df.head()

,languages,years
0,Python,NaN
1,Python,NaN
2,"Python, JavaScript, TypeScript, C, Swift",NaN
3,"Python, Java, JavaScript, R",1 - 5
4,Python,< 1


In [3]:
import re

# This regular expression checks if the string contains:
# - either the beginning of the string or a ", " delimeter
# - the name of the language (escaped because the + in C++ is a regex character)
# - either the end of the string or a ", " delimeter
# Note: the "?:" is to mark it as a 'match group' and is used to avoid warnings about captured groups
# See - https://stackoverflow.com/questions/39901550/python-userwarning-this-pattern-has-match-groups-to-actually-get-the-groups

for lang in all_langs:
    regex = rf'(?:^|, ){re.escape(lang)}(?:$|, )'
    df[lang] = df["languages"].str.contains(regex)
df

,languages,years,Python,Java,JavaScript,TypeScript,PHP,SQL,C,C++,...,Kotlin,Dart,Scala,Lua,Delphi,Haskell,Julia,Clojure,Elixir,Pascal
0,Python,NaN,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Python,NaN,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"Python, JavaScript, TypeScript, C, Swift",NaN,True,False,True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,"Python, Java, JavaScript, R",1 - 5,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Python,< 1,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,"Java, SQL, Swift",1 - 5,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
73,Java,15 - 20,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
74,"Python, JavaScript, C#",5 - 10,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
75,"Python, JavaScript, PHP, Ruby, Perl",15 - 20,True,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Find number of languages known
Print `"{# known by class} / {# in list} languages known by this class (as %)"`.

E.g. **12/21 languages known by this class (57%)**

In [4]:
known_langs = df.columns[df.eq(True).any()].tolist()
known_langs

['Python',
 'Java',
 'JavaScript',
 'TypeScript',
 'PHP',
 'SQL',
 'C',
 'C++',
 'C#',
 'Ruby',
 'R',
 'Matlab',
 'Go',
 'Rust',
 'Objective-C',
 'Swift',
 'Visual Basic',
 'Perl',
 'Cobol',
 'Fortran',
 'Lisp',
 'Assembly',
 'Kotlin',
 'Dart',
 'Lua',
 'Delphi',
 'Haskell',
 'Julia',
 'Clojure',
 'Elixir',
 'Pascal']

In [5]:
percent = round(len(known_langs)/len(all_langs) * 100)
print(f"{len(known_langs)}/{len(all_langs)} languages known by this class ({percent}%)")

31/32 languages known by this class (97%)


## List languages not known by anyone in the class

In [6]:
print('Not known:')
', '.join(df.columns[df.eq(False).all()].tolist())

Not known:


'Scala'

## Rank languages by most commonly known
Print each language as `"{position}: {language} ({percent_known}%)"`, in order from most to least known

e.g. **1: Python (93%)**

In [7]:
langs_count = df.iloc[:, 2:].sum()/len(df) * 100
langs_count

Python          81.818182
Java            46.753247
JavaScript      46.753247
TypeScript      11.688312
PHP             12.987013
SQL             53.246753
C               38.961039
C++             29.870130
C#              11.688312
Ruby             6.493506
R               10.389610
Matlab           2.597403
Go              10.389610
Rust             1.298701
Objective-C      3.896104
Swift            6.493506
Visual Basic    24.675325
Perl            19.480519
Cobol            6.493506
Fortran         12.987013
Lisp             6.493506
Assembly        16.883117
Kotlin           2.597403
Dart             1.298701
Scala            0.000000
Lua              1.298701
Delphi           6.493506
Haskell          1.298701
Julia            2.597403
Clojure          5.194805
Elixir           1.298701
Pascal          19.480519
dtype: float64

In [8]:
langs_count.sort_values(ascending=False, inplace=True)
langs_count

Python          81.818182
SQL             53.246753
JavaScript      46.753247
Java            46.753247
C               38.961039
C++             29.870130
Visual Basic    24.675325
Pascal          19.480519
Perl            19.480519
Assembly        16.883117
PHP             12.987013
Fortran         12.987013
C#              11.688312
TypeScript      11.688312
Go              10.389610
R               10.389610
Delphi           6.493506
Lisp             6.493506
Ruby             6.493506
Cobol            6.493506
Swift            6.493506
Clojure          5.194805
Objective-C      3.896104
Kotlin           2.597403
Julia            2.597403
Matlab           2.597403
Dart             1.298701
Lua              1.298701
Rust             1.298701
Haskell          1.298701
Elixir           1.298701
Scala            0.000000
dtype: float64

In [9]:
for i, (lang, count) in enumerate(langs_count.items(), start=1):
    print(f'{i}: {lang} ({round(count)}%)')

1: Python (82%)
2: SQL (53%)
3: JavaScript (47%)
4: Java (47%)
5: C (39%)
6: C++ (30%)
7: Visual Basic (25%)
8: Pascal (19%)
9: Perl (19%)
10: Assembly (17%)
11: PHP (13%)
12: Fortran (13%)
13: C# (12%)
14: TypeScript (12%)
15: Go (10%)
16: R (10%)
17: Delphi (6%)
18: Lisp (6%)
19: Ruby (6%)
20: Cobol (6%)
21: Swift (6%)
22: Clojure (5%)
23: Objective-C (4%)
24: Kotlin (3%)
25: Julia (3%)
26: Matlab (3%)
27: Dart (1%)
28: Lua (1%)
29: Rust (1%)
30: Haskell (1%)
31: Elixir (1%)
32: Scala (0%)


## Bonus: rank languages known by number of years programming


In [10]:
year_ranges = ['< 1', '1 - 5', '5 - 10', '10 - 15', '15 - 20', '> 20', 'Unknown']

df2 = df
df2['years'] = pd.Categorical(df['years'], categories=year_ranges)
df2['years'].fillna("Unknown", inplace=True)
df2['total'] = 1
df2.head()

,languages,years,Python,Java,JavaScript,TypeScript,PHP,SQL,C,C++,...,Dart,Scala,Lua,Delphi,Haskell,Julia,Clojure,Elixir,Pascal,total
0,Python,Unknown,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,Python,Unknown,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,"Python, JavaScript, TypeScript, C, Swift",Unknown,True,False,True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,1
3,"Python, Java, JavaScript, R",1 - 5,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4,Python,< 1,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1


In [11]:
by_years = df2.groupby('years').sum().drop(columns=['languages'])
by_years

,Python,Java,JavaScript,TypeScript,PHP,SQL,C,C++,C#,Ruby,...,Dart,Scala,Lua,Delphi,Haskell,Julia,Clojure,Elixir,Pascal,total
years,,,,,,,,,,,,,,,,,,,,,
< 1,6,1,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,6
1 - 5,19,11,12,1,1,16,4,3,1,0,...,0,0,0,0,0,0,1,0,2,22
5 - 10,4,0,3,0,0,3,2,2,1,0,...,0,0,0,0,0,0,0,0,0,8
10 - 15,10,8,4,1,1,3,7,7,3,2,...,0,0,0,2,0,0,0,0,4,12
15 - 20,6,5,5,3,2,4,2,0,1,1,...,0,0,0,1,0,0,0,0,1,8
> 20,14,11,11,3,6,15,13,10,3,2,...,1,0,1,1,1,2,3,1,7,17
Unknown,4,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [12]:
for index in by_years.index:
    num_respondents = by_years.loc[index, 'total']
    print(f"For {index} ({num_respondents})")
    if num_respondents == 0:
        print('  No data')
    else:
        new_df = by_years.drop(columns=['total'])
        print(new_df.loc[index, new_df.loc[index] >= 1].sort_values(ascending=False).to_string())
    print()

For < 1 (6)
Python    6
Java      1
C         1
C++       1
Swift     1
Delphi    1
Pascal    1

For 1 - 5 (22)
Python          19
SQL             16
JavaScript      12
Java            11
Perl             4
C                4
Visual Basic     4
C++              3
Assembly         2
R                2
Pascal           2
Go               1
Swift            1
C#               1
PHP              1
Cobol            1
TypeScript       1
Clojure          1

For 5 - 10 (8)
Python        4
JavaScript    3
SQL           3
C             2
C++           2
C#            1
Go            1
Cobol         1

For 10 - 15 (12)
Python          10
Java             8
C                7
C++              7
Visual Basic     5
R                4
JavaScript       4
Pascal           4
SQL              3
C#               3
Perl             3
Assembly         3
Ruby             2
Go               2
Fortran          2
Lisp             2
Delphi           2
PHP              1
TypeScript       1
Matlab           1

For

## Bonus: show the breakdown of each language by number of years programming

In [13]:
for lang in all_langs:
    new_df = by_years.loc[:, [lang, 'total']]
    percent = new_df.loc[:, lang] / new_df.loc[:, 'total'] * 100
    new_df['percent'] = percent.astype(int)
    print(new_df.to_string())
    print()

         Python  total  percent
years                          
< 1           6      6      100
1 - 5        19     22       86
5 - 10        4      8       50
10 - 15      10     12       83
15 - 20       6      8       75
> 20         14     17       82
Unknown       4      4      100

         Java  total  percent
years                        
< 1         1      6       16
1 - 5      11     22       50
5 - 10      0      8        0
10 - 15     8     12       66
15 - 20     5      8       62
> 20       11     17       64
Unknown     0      4        0

         JavaScript  total  percent
years                              
< 1               0      6        0
1 - 5            12     22       54
5 - 10            3      8       37
10 - 15           4     12       33
15 - 20           5      8       62
> 20             11     17       64
Unknown           1      4       25

         TypeScript  total  percent
years                              
< 1               0      6        0
1 - 5  